Leaderboard
===========

Basic analysis and stats for private leaderboards.

Private leaderboards are available at the url in the form:  
https://adventofcode.com/2021/leaderboard/private/view/{leaderboard_id}.json

### Instructions  
1) Visit your private leaderboard at the json link  
2) Save the json to the json folder  
3) Update the path_leaderboard to point to the leaderboard json file

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np

In [ ]:
path_leaderboard = 'json/1491207_20211206.json'
with open (path_leaderboard, 'r') as fh:
    data = json.load(fh)


In [ ]:
# Members
members = []
for member_id in data['members']:
    member_name = data['members'][member_id].get('name')
    member_local_score = data['members'][member_id].get('local_score')
    member = {
        "name": member_name,
        "id": member_id,
        "local_score": member_local_score
    }
    members.append(member)

df_members = pd.DataFrame.from_dict(members)
df_members.sort_values(['local_score'], ascending=False)

In [ ]:
member_count = df_members['id'].nunique()
member_count

In [ ]:
# Scores
scores = []

def time_in_seconds(dt):
    midnight = dt.replace(hour=5, minute=0, second=0, microsecond=0)
    seconds = (dt - midnight).seconds    
    return seconds

for member in data['members']:

    
    for day in data['members'][member].get('completion_day_level'):

        
        for star in data['members'][member].get('completion_day_level',{0: {}}).get(day, {}):
            score = ({
                'id': member, 
                'name': data['members'][member].get('name'),
                'day': int(day),
                'star': int(star),
                'ts': data['members'][member].get('completion_day_level').get(day).get(star)['get_star_ts'], 
                
            })
            score['dt'] = pd.to_datetime(score['ts'], unit='s')
            score['time_submitted'] = time_in_seconds(score['dt'])
            score['elapsed_time'] = pd.Timedelta(seconds=score['time_submitted'])
            scores.append(score)

df_scores = pd.DataFrame.from_dict(scores)
df_scores

In [ ]:
day_max = df_scores['day'].max()
day_max

In [ ]:
# Calculate points
df_scores["part_rank"] = df_scores.groupby(['day', 'star'])["ts"].rank("dense", ascending=True).astype('int')
df_scores["points"] = df_scores['part_rank'].apply(lambda x: (member_count + 1) - x).astype('int')


In [ ]:
# Cumulative points over time
df_scores['c_points'] = df_scores.sort_values(['ts']).groupby(['id'])['points'].cumsum()

### Reports

In [ ]:
# Latest Scores
df_scores[df_scores['day'] == day_max].sort_values(['dt'])

In [ ]:
# Leaderboard
df_scores.groupby(df_scores['name'])['c_points'].max().sort_values(ascending=False)

In [ ]:
# What time does a member usually submit Part 2 by?
df_scores[df_scores['star'] == 2].groupby(['name'])['time_submitted'].mean().apply(lambda x: pd.Timedelta(seconds=int(x) + 5 * 60 * 60)).sort_values()

In [ ]:
# On average, how long does a member take between submiting P1 and P2?
df_pt_stars = pd.pivot_table(df_scores, 
                             values='time_submitted', 
                             index=['name', 'day'],
                             columns=['star'], 
                             aggfunc=np.max, fill_value=0)

df_pt_stars['s_diff'] = df_pt_stars[(2)] - df_pt_stars[(1)]

df_p1_to_p2 = df_pt_stars[df_pt_stars[(2)]>0].groupby('name')['s_diff'].mean().sort_values().reset_index()
df_p1_to_p2['time'] = df_p1_to_p2['s_diff'].apply(lambda x:  pd.Timedelta(seconds=int(x)))
df_p1_to_p2